In [ ]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata

dataset = pd.read_csv("/content/movie_reviews (1).csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
def strip_html_tags(text):
    soup = BeautifulSoup(text,'html.parser')
    stripped_text = soup.get_text()
    stripper_text = re.sub(r'[\r|\n |\r\n]+','\n',stripped_text)
    return stripped_text


In [ ]:
strip_html_tags(dataset['review'][20])

"After the success of Die Hard and it's sequels it's no surprise really that in the 1990s, a glut of 'Die Hard on a .....' movies cashed in on the wrong guy, wrong place, wrong time concept. That is what they did with Cliffhanger, Die Hard on a mountain just in time to rescue Sly 'Stop or My Mom Will Shoot' Stallone's career.Cliffhanger is one big nit-pickers dream, especially to those who are expert at mountain climbing, base-jumping, aviation, facial expressions, acting skills. All in all it's full of excuses to dismiss the film as one overblown pile of junk. Stallone even managed to get out-acted by a horse! However, if you an forget all the nonsense, it's actually a very lovable and undeniably entertaining romp that delivers as plenty of thrills, and unintentionally, plenty of laughs.You've got to love John Lithgows sneery evilness, his tick every box band of baddies, and best of all, the permanently harassed and hapless 'turncoat' agent, Rex Linn as Travers.He may of been Henry in

In [ ]:
def remove_accented_character(text):
    text = unicodedata.normalize("NFKD",text).encode("ascii",'ignore').decode("utf-8","ignore")
    return text

In [ ]:
remove_accented_character("áü")

'au'

In [ ]:
def pre_process_corpus(texts):
    preprocessed_text = []
    for doc in tqdm.tqdm(texts):
      doc = strip_html_tags (doc)
      #doc = doc.translate(doc.maketrans("\n\t\r",'   '))
      doc = doc.lower()
      doc = remove_accented_character(doc)
      #doc = contractions.fix(doc)
      doc = re.sub(r'[^a-zA-Z0-9\s]','',doc,flags=re.I | re.A)
      doc = re.sub(r' +',' ',doc)
      doc = doc.strip()
      preprocessed_text.append(doc)
    return preprocessed_text

In [ ]:
reviews =  dataset['review'].values
sentiment =  dataset['sentiment'].values

train_reviews = reviews[:35000]
train_sentiment = sentiment[:35000]

test_reviews = reviews[35000:]
test_sentiment = sentiment[35000:]

In [ ]:
%%time
norm_train_reviews =  pre_process_corpus(train_reviews)
norm_test_reviews =  pre_process_corpus(test_reviews)

  2%|▏         | 725/35000 [00:00<00:41, 831.93it/s]<ipython-input-3-b62a4dd031b3>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text,'html.parser')
100%|██████████| 15000/15000 [00:05<00:00, 2562.69it/s]

CPU times: user 18.5 s, sys: 137 ms, total: 18.7 s
Wall time: 19.5 s


In [ ]:
import tensorflow as tf

t = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',oov_token='UNK')
t.fit_on_texts(norm_train_reviews)

In [ ]:
t.word_index['UNK']=0

In [ ]:
train_sequences = t.texts_to_sequences(norm_train_reviews)
test_sequences = t.texts_to_sequences(norm_test_reviews)


In [ ]:
len(t.word_index)

176885

In [ ]:
t.document_count

35000

In [ ]:
MAX_SEQUENCES_LENGTH = 1000

In [ ]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequences,maxlen=MAX_SEQUENCES_LENGTH)
X_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequences,maxlen=MAX_SEQUENCES_LENGTH)



In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
num_classes = 2


In [ ]:
y_train = le.fit_transform(train_sentiment)
y_test = le.transform(test_sentiment)

In [ ]:
VOCAB_SIZE = len(t.word_index)

In [ ]:
EMBEDDING_DIM = 300

In [ ]:
#GRU
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCES_LENGTH))
model.add(tf.keras.layers.GRU(128,return_sequences=False))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train,y_train,epochs=2,batch_size=100,shuffle = True, validation_data=(X_test,y_test))

Epoch 1/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 46s 108ms/step - accuracy: 0.7143 - loss: 0.5266 - val_accuracy: 0.8120 - val_loss: 0.4140
Epoch 2/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 40s 115ms/step - accuracy: 0.9197 - loss: 0.2116 - val_accuracy: 0.8734 - val_loss: 0.3077


In [ ]:
score = model.evaluate(X_test,y_test)
print("Accuracy:",score[1]*100)

469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.8741 - loss: 0.3043
Accuracy: 87.33999729156494


In [ ]:
embedding_layer = model.get_layer('embedding')

In [ ]:
embedding_weights = embedding_layer.get_weights()[0]

word_index = t.word_index
word_embeddings = {}

for word,idx in word_index.items():
  if idx<VOCAB_SIZE:
    word_embeddings[word] = embedding_weights[idx]

for word,embedding in list(word_embeddings.items())[10:15]:
  print(f'word: {word}, Embedding:{embedding}')



word: i, Embedding:[-0.007813   -0.03839122 -0.02245382  0.04700895  0.01180265  0.00884573
 -0.02104567  0.00222214 -0.0401674   0.01286882  0.02173615  0.00361161
 -0.03975101  0.02589889 -0.01370481  0.06798235 -0.00124058  0.05406816
  0.01229288 -0.01925366  0.03081059  0.03307713 -0.00862336  0.02949064
  0.02924531 -0.0016869  -0.00716808 -0.03275279 -0.03243267 -0.00656469
 -0.05033167  0.00638862  0.0291978   0.01767724  0.02526904 -0.01180799
  0.01260254  0.01943219  0.00065934 -0.01076648 -0.01460293  0.02512101
  0.00329463  0.05319835  0.01711995 -0.01266269  0.02729619  0.00234353
 -0.03700918  0.03110585 -0.04045486 -0.00053571  0.0156059  -0.00276183
  0.02065913 -0.05680316  0.01967493 -0.01495776 -0.00742954 -0.03570254
 -0.01050328  0.03672477 -0.03238372  0.02378994  0.03009538  0.00535012
  0.00288982 -0.0167408  -0.0073531   0.06380142  0.02247596 -0.01725754
  0.00122384 -0.00766461  0.01728735  0.02006915  0.02177977  0.00248529
 -0.03308154  0.00557651  0.0401

In [ ]:
#LSTM
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCES_LENGTH))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=False)))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])


In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train,y_train,epochs=2,batch_size=100,shuffle = True, validation_data=(X_test,y_test))

Epoch 1/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 81s 218ms/step - accuracy: 0.7485 - loss: 0.4930 - val_accuracy: 0.8827 - val_loss: 0.2892
Epoch 2/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 76s 216ms/step - accuracy: 0.9392 - loss: 0.1665 - val_accuracy: 0.8665 - val_loss: 0.3261


In [ ]:
score = model.evaluate(X_test,y_test)
print("Accuracy:",score[1]*100)

469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 31ms/step - accuracy: 0.8671 - loss: 0.3303
Accuracy: 86.64666414260864


In [ ]:
embedding_layer = model.get_layer('embedding_1')

In [ ]:
embedding_weights = embedding_layer.get_weights()[0]

word_index = t.word_index
word_embeddings = {}

for word,idx in word_index.items():
  if idx<VOCAB_SIZE:
    word_embeddings[word] = embedding_weights[idx]

for word,embedding in list(word_embeddings.items())[10:15]:
  print(f'word: {word}, Embedding:{embedding}')


word: i, Embedding:[ 2.51570288e-02  1.80154480e-02  4.18517925e-02  2.33383831e-02
 -2.71055894e-03 -5.39785139e-02  3.80656384e-02  2.61281477e-03
  2.18490791e-02 -5.62996529e-02  1.61722943e-03 -3.76769304e-02
 -1.86992995e-02 -2.80420370e-02  3.01398952e-02 -9.35219973e-03
 -1.19808159e-04  2.13077874e-03 -3.30117010e-02 -1.03905965e-02
  1.51420971e-02 -4.43628877e-02  8.70321505e-03  1.54806804e-02
 -7.10406946e-03  1.03589604e-02  3.46684866e-02 -3.96446250e-02
  1.43488841e-02  1.94258559e-02  3.82909132e-03 -3.59863937e-02
 -2.85783224e-02  7.41721317e-03 -1.62201021e-02 -1.81658920e-02
  2.64432114e-02 -6.45495020e-03  6.63709268e-02  2.00048033e-02
  1.20206121e-02 -3.07530202e-02 -4.53334115e-02 -3.60574268e-05
  3.78871895e-02 -2.16503106e-02  2.07957141e-02 -4.33195569e-02
 -1.86845362e-02  2.12609190e-02  1.06451930e-02 -2.78985798e-02
  1.65046975e-02  1.70317143e-02 -2.79089250e-02  1.49072381e-02
 -3.79507095e-02 -3.62046696e-02 -3.50061283e-02 -2.04049256e-02
  3.82

In [ ]:
#MULITIPLE LAYERS OF LSTM
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCES_LENGTH))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=False)))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train,y_train,epochs=2,batch_size=100,shuffle = True, validation_data=(X_test,y_test))

Epoch 1/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 238s 663ms/step - accuracy: 0.7377 - loss: 0.5077 - val_accuracy: 0.8145 - val_loss: 0.4030
Epoch 2/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 261s 661ms/step - accuracy: 0.9296 - loss: 0.1892 - val_accuracy: 0.8373 - val_loss: 0.3892


In [ ]:
score = model.evaluate(X_test,y_test)
print("Accuracy:",score[1]*100)

469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 89ms/step - accuracy: 0.8443 - loss: 0.3834
Accuracy: 83.73333215713501
